In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# /content/ASL

In [ ]:
import numpy
x_train = numpy.load('/content/x_train.npy')
y_train = numpy.load('/content/y_train.npy')

In [ ]:
!git clone https://github.com/ASLVA/ASL.git

fatal: destination path 'ASL' already exists and is not an empty directory.


In [1]:
# load train images and preprocess them using inbuilt functions

import cv2
import numpy
import glob

from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.utils import load_img, img_to_array

def fetch_image(file_path):
    path = glob.glob(file_path)
    imagenames_list = []
    image_labels = []
    for folder in path:
      for f in glob.glob(folder+"/*.*"):
        image_name = f.split('/')[6]
        image_labels.append(image_name)
        imagenames_list.append(f)

    print(len(imagenames_list))

    read_images = []     
    i = 0   
    for image in imagenames_list:
      i += 1
    
      read_images.append(load_img(image,color_mode='rgb', target_size=(224, 224)))
    return read_images, image_labels

In [21]:
# prepare x_train and y_train

import tensorflow as tf
def prepare_images(image_labels, read_images, test=None):
  image_data = []
  for i in range(len(read_images)):
    img_data = tf.keras.utils.img_to_array(read_images[i])
    # img_data = np.expand_dims(img_data, axis = 0)
    img_data = preprocess_input(img_data)

    image_data.append(img_data)
  x = numpy.asarray(image_data)

  print(x.shape)
  if test:
    y_train = []
    y_label = list(set(image_labels))
    for i in image_labels:
      y_train.append(y_label.index(f"{i}"))

    y = tf.keras.utils.to_categorical(y_train, num_classes=26) 
  else:
    y_train = []
    for i in range(len(image_labels)):
      y = ord(image_labels[i]) - 65
      y_train.append(y)

    y = tf.keras.utils.to_categorical(y_train, num_classes=26)

  return x, y

In [22]:
read_train_img, train_img_labels = fetch_image("/content/ASL/data/train_sample/train1/*")
x_train, y_train = prepare_images(train_img_labels, read_train_img)


260
(260, 224, 224, 3)


In [23]:
read_test_img, test_img_labels = fetch_image("/content/ASL/data/test_sample/test1/*")
x_test, y_test = prepare_images(test_img_labels, read_test_img, True)

260
(260, 224, 224, 3)


In [24]:
numpy.save('x_train', x_train)
numpy.save('y_train', y_train)

In [25]:
numpy.save('x_test', x_test)
numpy.save('y_test', y_test)

In [26]:
# split train dataset into train dataset and validation dataset

from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.3, random_state=42)

In [27]:
def fit(x,y,batch,epoch,x1,y1,model):
  model.fit(x, y, batch_size=batch, epochs=epoch, validation_data=(x1,y1))
  return model

In [28]:
# extract weights from all the layers in first model of the cascade

def get_weights(model):
  weights = []
  for layer in model.layers:
      wts = layer.get_weights()
      weights.append(wts)
  return weights

In [29]:
# Accuracy of model 1 in cascade 1, initialized with imagenet weights, trained on ASL dataset
import numpy as np

import warnings
warnings.filterwarnings('ignore')
# Accuracy
from sklearn import metrics
def eval_metrics(x,y,model):
  y_predicted = model.predict(x)
  metric = metrics.classification_report(np.argmax(y, axis=1), np.argmax(y_predicted, axis=1))
  return metric

In [ ]:
# import warnings
# warnings.filterwarnings('ignore')
# y_pred1 = eval_metrics(x_test,y_test,cascade_model_1a_gen)
# print(y_pred1)

In [31]:
def get_max_prob(y_pred):
  y_pred_list = []
  for i in y_pred:
    max_value = max(i)
    max_index = numpy.where(i == max_value)
    y_pred_list.append((max_value, max_index))

  return y_pred_list


First Cascade VGG16-VGG16

In [32]:
# add additional layers to the base model

CLASSES = 26
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.layers import Dense, Flatten
from keras import layers, models
from keras.callbacks import ModelCheckpoint
from keras.models import  load_model

def get_model(base_model):
  
  model = models.Sequential()
  model.add(base_model)
  model.add(layers.Flatten())
  model.add(layers.Dense(4096))  
  model.add(layers.Dense(CLASSES, activation='softmax'))

  model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])
  
  return model

In [33]:
# base model VGG1, Trained on ASL, Initialized with ImageNet

IMG_SIZE = (224,224)

def get_vgg():

  base_model_vgg = VGG16(
      weights='imagenet', 
      include_top=False, 
      input_shape=(224,224,3)
  )

  for layer in base_model_vgg.layers:
    layer.trainable = True

  return base_model_vgg

In [34]:
base_model_vgg = get_vgg()
cascade_vgg_1 = get_model(base_model_vgg)

In [ ]:
# # generalize this model to test on digits dataset, by training only the last layer freezing all other layers

# def get_gen_model(model):
  
#   model.pop()
#   model.add(layers.Dense(10, activation='softmax'))

#   model.compile(
#     loss='categorical_crossentropy',
#     optimizer='adam',
#     metrics=['accuracy'])
  
#   return model

In [ ]:
# cascade_model_1a_gen = get_gen_model(cascade_model_1a)

# for layer in cascade_model_1a_gen.layers:
#   if layer.output_shape == (None,10):
#     layer.trainable = True
#   else:
#     layer.trainable = False

# for layer in cascade_model_1a_gen.layers:
#   print(layer.name)
#   print(layer.trainable)

In [35]:
cascade_vgg_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 4096)              102764544 
                                                                 
 dense_1 (Dense)             (None, 26)                106522    
                                                                 
Total params: 117,585,754
Trainable params: 117,585,754
Non-trainable params: 0
_________________________________________________________________


In [36]:
cascade_vgg_1 = fit(x_train, y_train, 50, 10, x_valid,y_valid,cascade_vgg_1)

Epoch 1/10
4/4 [==============================] - 23s 4s/step - loss: 9.8620 - accuracy: 0.0385 - val_loss: 118.2850 - val_accuracy: 0.0385
Epoch 2/10
4/4 [==============================] - 3s 715ms/step - loss: 96.2342 - accuracy: 0.0330 - val_loss: 3.5240 - val_accuracy: 0.0513
Epoch 3/10
4/4 [==============================] - 3s 711ms/step - loss: 11.4036 - accuracy: 0.0330 - val_loss: 3.2768 - val_accuracy: 0.0385
Epoch 4/10
4/4 [==============================] - 3s 708ms/step - loss: 7.6945 - accuracy: 0.0275 - val_loss: 3.2701 - val_accuracy: 0.0385
Epoch 5/10
4/4 [==============================] - 3s 701ms/step - loss: 3.2567 - accuracy: 0.0385 - val_loss: 3.2727 - val_accuracy: 0.0256
Epoch 6/10
4/4 [==============================] - 3s 703ms/step - loss: 3.2564 - accuracy: 0.0440 - val_loss: 3.2752 - val_accuracy: 0.0256
Epoch 7/10
4/4 [==============================] - 3s 708ms/step - loss: 5.0453 - accuracy: 0.0330 - val_loss: 4.3571 - val_accuracy: 0.0128
Epoch 8/10
4/4 [==

In [40]:
y_pred_vgg1 = cascade_vgg_1.predict(x_test)

In [41]:
y_pred_vgg1 

In [38]:
y_pred_vgg1 = get_max_prob(y_pred_vgg1)

In [39]:
y_pred_vgg1

[(0.06993494, (array([3]),)),
 (0.07075192, (array([3]),)),
 (0.069383055, (array([3]),)),
 (0.069329455, (array([3]),)),
 (0.0691614, (array([3]),)),
 (0.07006709, (array([3]),)),
 (0.07029978, (array([3]),)),
 (0.06882346, (array([3]),)),
 (0.06960281, (array([3]),)),
 (0.069446914, (array([3]),)),
 (0.0700002, (array([3]),)),
 (0.06951658, (array([3]),)),
 (0.06983363, (array([3]),)),
 (0.06871044, (array([3]),)),
 (0.072325744, (array([3]),)),
 (0.07086916, (array([3]),)),
 (0.07056249, (array([3]),)),
 (0.06986312, (array([3]),)),
 (0.070327066, (array([3]),)),
 (0.069809385, (array([3]),)),
 (0.06123485, (array([3]),)),
 (0.06749503, (array([3]),)),
 (0.068847135, (array([3]),)),
 (0.06019421, (array([3]),)),
 (0.06674305, (array([3]),)),
 (0.06570539, (array([3]),)),
 (0.06334678, (array([3]),)),
 (0.06736229, (array([3]),)),
 (0.0639494, (array([3]),)),
 (0.06699513, (array([3]),)),
 (0.07076482, (array([3]),)),
 (0.0695328, (array([3]),)),
 (0.07301694, (array([3]),)),
 (0.070

In [ ]:
wts_cascade_vgg = get_weights(cascade_vgg_1)

In [ ]:
# train the second model VGG in the cascade with the weights set from the first model VGG
# base model VGG2, Trained on ASL, Initialized with ASL weights

base_model_vgg = get_vgg()

## iterate through base model layer names:

layer_count = 0
for layer in base_model_vgg.layers:
  if "pool" not in layer.name and "input" not in layer.name:
    weights = wts_cascade_vgg[0].pop(0)
    bias = wts_cascade_vgg[0].pop(0)

    # set weights and biases for base model 1
    base_model_vgg.layers[layer_count].set_weights([weights,bias])
  layer_count += 1

cascade_vgg_2 = get_model(base_model_vgg)

In [ ]:
cascade_vgg_2 = fit(x_train, y_train, 50, 10, x_valid,y_valid,cascade_vgg_2)

Epoch 1/10
4/4 [==============================] - 4s 862ms/step - loss: 3.2612 - accuracy: 0.0165 - val_loss: 3.2644 - val_accuracy: 0.0256
Epoch 2/10
4/4 [==============================] - 3s 803ms/step - loss: 3.2571 - accuracy: 0.0385 - val_loss: 3.2728 - val_accuracy: 0.0128
Epoch 3/10
4/4 [==============================] - 3s 815ms/step - loss: 3.2564 - accuracy: 0.0385 - val_loss: 3.2798 - val_accuracy: 0.0128
Epoch 4/10
4/4 [==============================] - 3s 813ms/step - loss: 3.2511 - accuracy: 0.0495 - val_loss: 3.2892 - val_accuracy: 0.0128
Epoch 5/10
4/4 [==============================] - 3s 825ms/step - loss: 11.3927 - accuracy: 0.0495 - val_loss: 3.2923 - val_accuracy: 0.0128
Epoch 6/10
4/4 [==============================] - 3s 822ms/step - loss: 3.2488 - accuracy: 0.0385 - val_loss: 3.2968 - val_accuracy: 0.0128
Epoch 7/10
4/4 [==============================] - 3s 832ms/step - loss: 3.2484 - accuracy: 0.0495 - val_loss: 3.3026 - val_accuracy: 0.0128
Epoch 8/10
4/4 [===

In [ ]:
import warnings
warnings.filterwarnings('ignore')
y_pred2 = eval_metrics(x_test,y_test,cascade_model_1b)
print(y_pred2)

4/4 [==============================] - 1s 157ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.00      0.00      0.00        10
           2       0.00      0.00      0.00        10
           3       0.00      0.00      0.00        10
           4       0.00      0.00      0.00        10
           5       0.00      0.00      0.00        10
           6       0.00      0.00      0.00        10
           7       0.00      0.00      0.00        10
           8       0.10      1.00      0.19        10
           9       0.00      0.00      0.00        10
          19       0.00      0.00      0.00         0
          25       0.00      0.00      0.00         0

    accuracy                           0.10       100
   macro avg       0.01      0.08      0.02       100
weighted avg       0.01      0.10      0.02       100



In [ ]:
# cascade_model_1b.layers[0].get_weights()

In [ ]:
y_pred1 = cascade_model_1b.predict(x_test)
classes_x=np.argmax(y_pred1,axis=1)

4/4 [==============================] - 1s 168ms/step


In [ ]:
# #[[[vgg_values]],[flatten],[Dense],[softmax]]
# len(weights_list[0])  #----> len(VGG)
# len(weights_list[0][0]) # First layer of VGG shape --> (3,3,3,64)  --- > channels R, G, B
# #3 
# len(weights_list[0][0][0]) # First layer of VGG shape --> (3,3,3,64)  --- > channel R, last index if it changes to 1 gives G, if its 2 it gives B

In [ ]:
##### R values #####
# []
# 1st sublist[R] - 1st sublist = first row first column of all 64 filters
# 1st sublist[R] - 2nd sublist = first row, second column of all 64 filters
# 1st sublist[R]  - 3rd sublist = first row ,third column of all 64 filters

In [ ]:
# []
# 2st sublist[R] - 1st sublist = second row first column of all 64 filters
# 2st sublist[R] - 2nd sublist = second row, second column of all 64 filters
# 2st sublist[R]  - 3rd sublist = second row ,third column of all 64 filters

In [ ]:
# []
# 3st sublist[R] - 1st sublist = third row first column of all 64 filters
# 3st sublist[R] - 2nd sublist = third row, second column of all 64 filters
# 3st sublist[R]  - 3rd sublist = third row ,third column of all 64 filters

In [ ]:
# First row of the R channels filters 
# weights_list[0][0][0][0]

In [ ]:
# first sublist is R, second sublist is G and third sublist is B, 
# each sublist within shows 1st row, 2nd row and 3rd row

# weights_1 = weights_list[0][0]
# weights_1

In [ ]:
## model [0][0] - 1st layer weights      ## base_model_1 [0][0] - inputs
## model [0][1] - 1st layer bias         ## base_model_1 [0][1] - 1st layer weights + bias,

## model [0][2] - 2nd layer weights      ## base_model_1 [0][2] - 2nd layer weights + bias, 2,3
## model [0][3] - 2nd layer bias         
                                         ## base_model_1 [0][3] - maxpool

## model [0][4] - 3rd layer weights      ## base_model_1 [0][4] - 3rd layer weights + bias, 4,5
## model [0][5] - 3rd layer bias         

## model [0][6] - 4th layer weights      ## base_model_1 [0][4] - 4th layer weights + bias, 6,7
## model [0][7] - 4th layer bias 

## model [0][8] - 4th layer weights      ## base_model_1 [0][5] - 4th layer weights + bias, 8,9
## model [0][9] - 4th layer bias 

In [ ]:
## base_model [0][0] - 1st layer weights      ## base_model_1 [0][0] - inputs
## base_model [0][1] - 1st layer bias         ## base_model_1 [0][1] - 1st layer weights + bias, 0,1

## base_model [0][2] - 2nd layer weights      ## base_model_1 [0][2] - 2nd layer weights + bias, 2,3
## base_model [0][3] - 2nd layer bias         

## base_model [0][4] - 3rd layer weights      ## base_model_1 [0][3] - 3rd layer weights + bias, 4,5
## base_model [0][5] - 3rd layer bias         

## base_model [0][6] - 4th layer weights      ## base_model_1 [0][4] - 4th layer weights + bias, 6,7
## base_model [0][7] - 4th layer bias 

## base_model [0][8] - 4th layer weights      ## base_model_1 [0][5] - 4th layer weights + bias, 8,9
## base_model [0][9] - 4th layer bias 

In [ ]:
## <<<<<<<<<<<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>>>>>>>>>>>

In [ ]:
# test_model = VGG16(
#     weights='imagenet', 
#     include_top=False, 
#     input_shape=(224,224,3)
# )

In [ ]:
# import numpy as np
# import random
# test_weights = np.random.rand(3,3,3,64)
# test_bias = np.random.rand(64)

In [ ]:
# test_weights

In [ ]:
# test_weights[:,:,:,0]

In [ ]:
# layer = test_model.layers[1]
# t_weights, t_bias = layer.get_weights()

In [ ]:
# test_model.layers[1].set_weights([test_weights,test_bias])

In [ ]:
# layer = test_model.layers[1]
# t1_weights, t1_bias = layer.get_weights()

In [ ]:
# t1_weights[:,:,:,0]

Second Cascade - MobileNet-MobileNet

In [ ]:
from keras.applications.mobilenet import MobileNet

def get_mnet():
  base_model_mnet = MobileNet(
      weights='imagenet', 
      include_top=False, 
      input_shape=(224,224,3)
  )

  for layer in base_model_mnet.layers:
    layer.trainable = True

  return base_model_mnet


In [ ]:
## build second base model MobileNet

base_model_mnet = get_mnet()
cascade_mnet_1 = get_model(base_model_mnet)

17235968/17225924 [==============================] - 0s 0us/step


In [ ]:
# cascade_model_2a.summary()

In [ ]:
cascade_mnet_1 = fit(x_train, y_train, 50, 10, x_valid, y_valid,cascade_mnet_1)

Epoch 1/10
4/4 [==============================] - 7s 919ms/step - loss: 184.7429 - accuracy: 0.0879 - val_loss: 1050.3053 - val_accuracy: 0.0897
Epoch 2/10
4/4 [==============================] - 1s 348ms/step - loss: 190.3318 - accuracy: 0.5879 - val_loss: 1291.9453 - val_accuracy: 0.0385
Epoch 3/10
4/4 [==============================] - 1s 348ms/step - loss: 38.7932 - accuracy: 0.7473 - val_loss: 1162.2460 - val_accuracy: 0.1795
Epoch 4/10
4/4 [==============================] - 1s 346ms/step - loss: 13.3692 - accuracy: 0.8956 - val_loss: 1645.5734 - val_accuracy: 0.1154
Epoch 5/10
4/4 [==============================] - 1s 343ms/step - loss: 5.1985 - accuracy: 0.9396 - val_loss: 1580.4072 - val_accuracy: 0.1154
Epoch 6/10
4/4 [==============================] - 1s 332ms/step - loss: 4.2585 - accuracy: 0.9396 - val_loss: 1433.5363 - val_accuracy: 0.0897
Epoch 7/10
4/4 [==============================] - 1s 347ms/step - loss: 3.6778 - accuracy: 0.9615 - val_loss: 1296.1169 - val_accuracy: 

In [ ]:
wts_cascade_mnet = get_weights(cascade_mnet_1)

In [ ]:
# train the second model VGG in the cascade with the weights set from the first model VGG
# base model VGG2, Trained on ASL, Initialized with ASL weights

base_model_mnet = get_mnet()

layer_count = 0
for layer in base_model_mnet.layers:
  if "pool" not in layer.name and "input" not in layer.name and "pad" not in layer.name and "relu" not in layer.name:
    if "bn" in layer.name:
      bn_weights = []
      for i in range(0,4):
        bn_weights.append(wts_cascade_mnet[0].pop(0))
      base_model_mnet.layers[layer_count].set_weights(bn_weights)
    else:
      weights = wts_cascade_mnet[0].pop(0)
      base_model_mnet.layers[layer_count].set_weights([weights])
  layer_count += 1

cascade_mnet_2 = get_model(base_model_mnet)

In [ ]:
cascade_mnet_2= fit(x_train, y_train, 50, 10, x_valid, y_valid,cascade_mnet_2)

Epoch 1/10
4/4 [==============================] - 4s 508ms/step - loss: 56.8640 - accuracy: 0.2582 - val_loss: 1489.9406 - val_accuracy: 0.1410
Epoch 2/10
4/4 [==============================] - 1s 345ms/step - loss: 4.1550 - accuracy: 0.7967 - val_loss: 2389.0132 - val_accuracy: 0.0897
Epoch 3/10
4/4 [==============================] - 1s 349ms/step - loss: 1.8284 - accuracy: 0.8462 - val_loss: 2739.8589 - val_accuracy: 0.0897
Epoch 4/10
4/4 [==============================] - 1s 358ms/step - loss: 1.2381 - accuracy: 0.8846 - val_loss: 2844.4558 - val_accuracy: 0.1026
Epoch 5/10
4/4 [==============================] - 1s 347ms/step - loss: 0.6225 - accuracy: 0.9011 - val_loss: 2847.6646 - val_accuracy: 0.1282
Epoch 6/10
4/4 [==============================] - 1s 356ms/step - loss: 0.3505 - accuracy: 0.9286 - val_loss: 3025.5244 - val_accuracy: 0.0897
Epoch 7/10
4/4 [==============================] - 1s 360ms/step - loss: 0.3964 - accuracy: 0.9615 - val_loss: 3091.8306 - val_accuracy: 0.076

Preprocess Test Dataset

In [ ]:
## preprocess test data

import cv2
import numpy
import glob

from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.utils import load_img, img_to_array

path = glob.glob("/content/drive/MyDrive/NN DL Project/dataset/test1/*")

test_list = []
test_labels = []
for folder in path:
  for f in glob.glob(folder+"/*.JPG"):
    test_labels.append(folder[-1])
    test_list.append(f)

print(len(test_list))

test_images = []     
i = 0   
for image in test_list:
  i += 1
  test_images.append(load_img(image,color_mode='rgb', target_size=(224, 224)))

100


In [ ]:
# prepare x_test and y_test

import tensorflow as tf
test_data = []
for i in range(len(test_images)):
  img_data = tf.keras.utils.img_to_array(read_images[i])
  # img_data = np.expand_dims(img_data, axis = 0)
  img_data = preprocess_input(img_data)

  test_data.append(img_data)

x_test = numpy.asarray(test_data)

y_test = test_labels
# for i in range(len(test_labels)):
#   y = ord(test_labels[i]) - 65
#   y_test.append(y)

y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

Third Cascade - ResNet50-ResNet50

In [ ]:
# train third pre-trained model Resnet
from keras.applications.resnet import ResNet50

def get_resNet():
  base_model_res = ResNet50(
      weights='imagenet', 
      include_top=False, 
      input_shape=(224,224,3)
  )

  for layer in base_model_res.layers:
    layer.trainable = True
  
  return base_model_res

In [ ]:
base_model_resnet = get_resNet()
cascade_resnet_1 = get_model(base_model_resnet)

In [ ]:
cascade_resnet_1 = fit(x_train, y_train, 20, 1, x_valid, y_valid, cascade_resnet_1)

10/10 [==============================] - 15s 607ms/step - loss: 255.8025 - accuracy: 0.0769 - val_loss: 11857.8652 - val_accuracy: 0.0641


In [ ]:
wts_cascade_resnet = get_weights(cascade_resnet_1)

In [ ]:
# train the second model VGG in the cascade with the weights set from the first model VGG
# base model VGG2, Trained on ASL, Initialized with ASL weights

base_model_resnet = get_resNet()



layer_count = 0
for layer in base_model_resnet.layers:
  if "pool" not in layer.name and "input" not in layer.name and "pad" not in layer.name and "relu" not in layer.name and "add" not in layer.name and "out" not in layer.name:
    if "bn" in layer.name:
      bn_weights = []
      for i in range(0,4):
        bn_weights.append(wts_cascade_resnet[0].pop(0))
      base_model_resnet.layers[layer_count].set_weights(bn_weights)
    else:
      weights = wts_cascade_resnet[0].pop(0)
      bias = wts_cascade_resnet[0].pop(0)
      base_model_resnet.layers[layer_count].set_weights([weights,bias])
  layer_count += 1

cascade_resnet_2 = get_model(base_model_resnet)

In [ ]:
cascade_resnet_2 = fit(x_train, y_train, 20, 1, x_valid, y_valid, cascade_resnet_2)

10/10 [==============================] - 9s 420ms/step - loss: 27.6641 - accuracy: 0.0330 - val_loss: 20635270.0000 - val_accuracy: 0.0128


In [ ]:
y_pred3 = cascade_resnet_2.predict(x_test)

NameError: ignored